## Scraping Wealthsimple Dividends with Scrapy

In [2]:
# todo: import the scrapy module
from scrapy import Selector
from datetime import datetime
import pandas as pd

    - get the html data first using the javascript code in the info file.

In [3]:
# use with open() as f: to open the html file
with open('wealthsimple_scraped.html', 'r') as f:
    html = f.read()

sel = Selector(text=html)    

In [4]:
# div_xpath = '//*[contains(@class,"kFJHYd")]'

symbol_xpath = '//*[@class="sc-204449cf-0 jUrmPc"]/text()'
date_xpath = '//*[@class="sc-204449cf-0 bIIoGo"]/text()'
payout_xpath = '//*[@class="sc-204449cf-0 bgBEGg"]/text()'

In [5]:
#  Using xpath to extract the data
print(sel.xpath(symbol_xpath).extract_first())
print(sel.xpath(date_xpath).extract_first())
print(sel.xpath(payout_xpath).extract_first())

GNK
TFSA | Nov 30, 2023
$0.90 USD


In [6]:
print(sel.xpath(symbol_xpath).extract())
print(sel.xpath(date_xpath).extract())
print(sel.xpath(payout_xpath).extract())

['GNK', 'CIF', 'CLM', 'GOF', 'AVK', 'GIM', 'FAX', 'TEI', 'OXSQ', 'FCO', 'PTMN', 'CLM', 'ARR', 'ARR', 'ZTR', 'CVI', 'HAFC', 'LGI', 'PSEC', 'CHI', 'CVI', 'MSB', 'UAN', 'OPI', 'CALM', 'HAS', 'ALLY', 'SLG', 'CFG', 'SJT', 'ETRN', 'BMRC', 'MITT', 'AGNC', 'PTR', 'DMLP', 'PFLT', 'EMD', 'MFV', 'CLM']
['TFSA | Nov 30, 2023', 'TFSA | Nov 30, 2023', 'TFSA | Nov 30, 2023', 'TFSA | Nov 30, 2023', 'TFSA | Nov 30, 2023', 'TFSA | Nov 30, 2023', 'TFSA | Nov 30, 2023', 'TFSA | Nov 30, 2023', 'TFSA | Nov 30, 2023', 'TFSA | Nov 30, 2023', 'TFSA | Nov 30, 2023', 'RRSP | Nov 30, 2023', 'TFSA | Nov 29, 2023', 'RRSP | Nov 29, 2023', 'TFSA | Nov 24, 2023', 'TFSA | Nov 22, 2023', 'TFSA | Nov 22, 2023', 'TFSA | Nov 22, 2023', 'TFSA | Nov 21, 2023', 'TFSA | Nov 20, 2023', 'TFSA | Nov 20, 2023', 'TFSA | Nov 20, 2023', 'TFSA | Nov 20, 2023', 'TFSA | Nov 16, 2023', 'TFSA | Nov 16, 2023', 'TFSA | Nov 15, 2023', 'TFSA | Nov 15, 2023', 'TFSA | Nov 15, 2023', 'TFSA | Nov 15, 2023', 'TFSA | Nov 14, 2023', 'TFSA | Nov 14, 

In [7]:
#  create a function that takes the date list and returns a list of datetime objects
def convert_to_datetime(date_list):
    account_list = []
    new_date_list = []
    for item in date_list:
        splice = item.split(sep='|')
        date_string = splice[1].strip()
        account_string = splice[0].strip()
        date_object = datetime.strptime(date_string, '%b %d, %Y')
        date_object = date_object.date()
        new_date_list.append(date_object)
        account_list.append(account_string)
    return account_list, new_date_list

In [8]:
symbol_list = sel.xpath(symbol_xpath).extract()
date_list = sel.xpath(date_xpath).extract()
account_list, date_list = convert_to_datetime(date_list)
payout_list = sel.xpath(payout_xpath).extract()

# zip the lists together
dividends_paid_list = list(zip(symbol_list, account_list, date_list, payout_list))

In [9]:
#  convert to dataframe
df = pd.DataFrame(dividends_paid_list, columns=['symbol', 'type', 'date', 'payout'])
# put the date column into this format: Aug 17, 2023
df['date'] = pd.to_datetime(df['date']).dt.strftime('%b %d, %Y')
# create a column for the month
df['month'] = pd.to_datetime(df['date']).dt.strftime('%b')
# create a column for the year
df['year'] = pd.to_datetime(df['date']).dt.strftime('%Y')
#  move the payout column to the end
cols = list(df.columns.values)
cols.pop(cols.index('payout'))
df = df[cols+['payout']]
df['payout'] = df['payout'].str.replace(' USD', '')

df

,symbol,type,date,month,year,payout
0,GNK,TFSA,"Nov 30, 2023",Nov,2023,$0.90
1,CIF,TFSA,"Nov 30, 2023",Nov,2023,$0.82
2,CLM,TFSA,"Nov 30, 2023",Nov,2023,$1.23
3,GOF,TFSA,"Nov 30, 2023",Nov,2023,$1.09
4,AVK,TFSA,"Nov 30, 2023",Nov,2023,$0.82
5,GIM,TFSA,"Nov 30, 2023",Nov,2023,$0.59
6,FAX,TFSA,"Nov 30, 2023",Nov,2023,$1.02
7,TEI,TFSA,"Nov 30, 2023",Nov,2023,$0.79
8,OXSQ,TFSA,"Nov 30, 2023",Nov,2023,$0.95
9,FCO,TFSA,"Nov 30, 2023",Nov,2023,$1.12


In [10]:
#  save the dataframe to a csv file
df.to_csv('wealthsimple_dividends.csv', index=False)

## Create a new dataframe with only symbols and payout so I can add them up and append to the UFX Dashboard  

In [21]:
# read the wealthsimple_dividends.csv file
wealthsimple_df = pd.read_csv('wealthsimple_dividends.csv')

In [22]:
# keep only sumbol and payout columns
wealthsimple_df = wealthsimple_df[['type', 'symbol', 'payout']]

# get rid of $ and convert the payout column to float
wealthsimple_df['payout'] = wealthsimple_df['payout'].str.replace('$', '', regex=False)
wealthsimple_df['payout'] = wealthsimple_df['payout'].astype(float)

# sort the dataframe by symbol
wealthsimple_df = wealthsimple_df.sort_values(by=['symbol'])
# reset the index
wealthsimple_df = wealthsimple_df.reset_index(drop=True)

wealthsimple_df

,type,symbol,payout
0,TFSA,AGNC,0.60
1,TFSA,ALLY,1.20
2,TFSA,ARR,0.64
3,RRSP,ARR,1.12
4,TFSA,AVK,0.82
5,TFSA,BMRC,1.25
6,TFSA,CALM,0.01
7,TFSA,CFG,1.68
8,TFSA,CHI,0.76
9,TFSA,CIF,0.82


In [33]:
# read in the current_holdings full sheet from dividend dashboard

dividend_dashboard_df = pd.read_excel(r'C:\\Dividend_Projects\\DIVIDEND DASHBOARD\\data\\Dividend_Dashboard.xlsx', sheet_name='current_holdings')

# keep only the followign columns: Ticker, Div. Earned	Acct. Type
dividend_dashboard_df = dividend_dashboard_df[['Ticker', 'Div. Earned', 'Acct. Type']]

# change Nan to 0 in the Div. Earned column 
dividend_dashboard_df['Div. Earned'] = dividend_dashboard_df['Div. Earned'].fillna(0)


In [35]:
dividend_dashboard_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Ticker       68 non-null     object 
 1   Div. Earned  68 non-null     float64
 2   Acct. Type   68 non-null     object 
dtypes: float64(1), object(2)
memory usage: 1.7+ KB


In [40]:
wealthsimple_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   type    36 non-null     object 
 1   symbol  36 non-null     object 
 2   payout  36 non-null     float64
dtypes: float64(1), object(2)
memory usage: 992.0+ bytes


In [39]:
# check for duplicates in wealthsimple dataframe where the symbol and type are the same
wealthsimple_df.duplicated(subset=['symbol', 'type']).sum()

# print the duplicates
wealthsimple_df[wealthsimple_df.duplicated(subset=['symbol', 'type'], keep=False)]


,type,symbol,payout
12,TFSA,CVI,4.5
13,TFSA,CVI,1.5


In [44]:
# Checking for duplicates where 'symbol' and 'type' are the same
duplicates_count = wealthsimple_df.duplicated(subset=['symbol', 'type']).sum()

# Printing the duplicates
duplicates_df = wealthsimple_df[wealthsimple_df.duplicated(subset=['symbol', 'type'], keep=False)]

# Grouping the DataFrame by 'symbol' and 'type' and summing the 'payout'
grouped_df = wealthsimple_df.groupby(['type', 'symbol']).sum().reset_index()

# print out the duplicates count, duplicates dataframe, and grouped dataframe in a nice formated information with a space and header
print(f'''
Duplicates Count: {duplicates_count}\n
Duplicates DataFrame:
{duplicates_df}\n
Grouped DataFrame: 
{grouped_df}\n
''')

# duplicates_count
# duplicates_df
# grouped_df



Duplicates Count: 1

Duplicates DataFrame:
    type symbol  payout
12  TFSA    CVI     4.5
13  TFSA    CVI     1.5

Grouped DataFrame: 
    type symbol  payout
0   RRSP    ARR    1.12
1   RRSP    CLM    0.86
2   TFSA   AGNC    0.60
3   TFSA   ALLY    1.20
4   TFSA    ARR    0.64
5   TFSA    AVK    0.82
6   TFSA   BMRC    1.25
7   TFSA   CALM    0.01
8   TFSA    CFG    1.68
9   TFSA    CHI    0.76
10  TFSA    CIF    0.82
11  TFSA    CLM    1.23
12  TFSA    CVI    6.00
13  TFSA   DMLP    2.54
14  TFSA   ETRN    1.65
15  TFSA    FAX    1.02
16  TFSA    FCO    1.12
17  TFSA    GIM    0.59
18  TFSA    GNK    0.90
19  TFSA    GOF    1.09
20  TFSA   HAFC    1.50
21  TFSA    HAS    1.40
22  TFSA    LGI    0.56
23  TFSA   MITT    1.52
24  TFSA    MSB    1.40
25  TFSA    OPI    1.25
26  TFSA   OXSQ    0.95
27  TFSA   PSEC    1.02
28  TFSA   PTMN    2.76
29  TFSA    PTR    5.22
30  TFSA    SJT    0.34
31  TFSA    SLG    0.81
32  TFSA    TEI    0.79
33  TFSA    UAN    1.55
34  TFSA    ZTR    0.60

In [47]:
grouped_df.head(1)

,type,symbol,payout
0,RRSP,ARR,1.12


In [48]:
# Merge the dataframes on 'symbol' and 'Ticker'
# Merge the dataframes on 'Ticker' and 'symbol', as well as 'type'
merged_df = dividend_dashboard_df.merge(grouped_df, left_on=['Ticker', 'Acct. Type'], right_on=['symbol', 'type'], how='left')

merged_df


,Ticker,Div. Earned,Acct. Type,type,symbol,payout
0,ETRN,0.00,TFSA,TFSA,ETRN,1.65
1,EURN,0.00,TFSA,NaN,NaN,NaN
2,BKCC,12.50,TFSA,NaN,NaN,NaN
3,F,0.00,TFSA,NaN,NaN,NaN
4,GLD,0.00,RRSP,NaN,NaN,NaN
...,...,...,...,...,...,...
63,IEP,10.00,TFSA,NaN,NaN,NaN
64,VIA,2.72,TFSA,NaN,NaN,NaN
65,MPW,11.13,RRSP,NaN,NaN,NaN
66,PTR,11.10,TFSA,TFSA,PTR,5.22


In [53]:
# change Nan to 0 in the payout column
merged_df['payout'] = merged_df['payout'].fillna(0)
# drop type	symbol columns
merged_df = merged_df.drop(columns=['type', 'symbol'])
# create a new column called 'Total Dividends'
merged_df['Total Dividends'] = merged_df['Div. Earned'] + merged_df['payout']
merged_df

,Ticker,Div. Earned,Acct. Type,payout,Total Dividends
0,ETRN,0.00,TFSA,1.65,1.65
1,EURN,0.00,TFSA,0.00,0.00
2,BKCC,12.50,TFSA,0.00,12.50
3,F,0.00,TFSA,0.00,0.00
4,GLD,0.00,RRSP,0.00,0.00
...,...,...,...,...,...
63,IEP,10.00,TFSA,0.00,10.00
64,VIA,2.72,TFSA,0.00,2.72
65,MPW,11.13,RRSP,0.00,11.13
66,PTR,11.10,TFSA,5.22,16.32


In [18]:
# save to dividends_tallied.csv
merged_df.to_csv('dividends_tallied.csv', index=False)